# ANALYSIS

In [ ]:
import pandas as pd
import ast
import numpy as np

# Load the csv file

In [ ]:
# df = pd.read_csv('best-books.csv',index_col='book_index')
df = pd.read_csv('best-books_old.csv')

# Change the awards col into num of awards:

In [ ]:
df = df.replace({'awards': {np.nan: 0}})

d = []

for i in df.index:
    if df.loc[i, "awards"] == 0:
        d.append(0)
    else:
        d.append( len( ast.literal_eval(df.loc[i, "awards"]) ) )

df['awards length'] = d
# df['awards length'].sum()

# min max normalization

In [ ]:
df_min_max = df['avg_rating'].copy()
def min_max_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.min()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

minmax_norm_ratings = min_max_normalization(df_min_max)
# print(minmax_norm_ratings)

df['minmax_norm_ratings'] = minmax_norm_ratings
# print(df)

# mean normalization

In [ ]:
def mean_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.mean()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

mean_norm_ratings = mean_normalization(df_min_max)

df['mean_norm_ratings'] = mean_norm_ratings
# print(df)

# sort by minmax_norm_ratings Given Author

In [ ]:
def check_author(df, author):
    df = df.loc[df.author == author]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

# check_author(df, 'J.K. Rowling')

# sort by publishing year and get mean of minmax_norm_ratings

In [ ]:
df.groupby('original_publish_year')['minmax_norm_ratings'].mean()

# sort by author and check who has highest average rating

In [ ]:
df.groupby('author')['minmax_norm_ratings'].mean().sort_values( ascending=False)

# Sort by author and total no of awards

In [ ]:
df.groupby('author')['awards length'].sum().sort_values( ascending=False)
# df.sort_values(by=['awards length', 'author'], ascending=[False, True])

# sort by minmax_norm_ratings Given Year of publication

In [ ]:
def check_year(df, year):
    df = df.loc[df.original_publish_year == year]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

# check_year(df, 2005)

# check if book series are rated better than non series books + the total no of books of each type

In [ ]:
df.groupby('series')['minmax_norm_ratings'].mean()
df.groupby('series').size()

# compare no of ratings with average ratings

In [ ]:
df[ (df['num_ratings'] < df['num_ratings'].std()) & (df['avg_rating']>4) ].sort_values(by=['num_ratings'], ascending=[True])

# clean data in PLACES column

In [ ]:
# convert the type of data
# df = df.replace({'places': {np.nan: 0}})
# dictt = []
# for i in df.index:
#     if df.loc[i, "places"] == 0:
#         dictt.append(0)
#     else:
#         dictt.append( ast.literal_eval(df.loc[i, "places"]) )
# df['places_new'] = dictt
# print(df['places_new'])

In [234]:
df = df.replace({'places': {np.nan: ['Ignore this']}})

# df2['places'] = df2['places'].apply(eval)
def to_1D(series):
    return pd.Series([x for x in series])

to_1D(df["places"]).value_counts()

Ignore this                                                          370
['The United States of America']                                      33
['London, England']                                                   14
['New York City, New York']                                           11
['United Kingdom']                                                    11
                                                                    ... 
['London, England', 'Woolton, Hertfordshire, England', 'England']      1
['Oxshott, Surrey, England', 'London, England']                        1
['New Hampshire', 'Vienna', 'New York City, New York']                 1
['Luna', 'New Beijing, Eastern Commonwealth', 'Sahara']                1
['New York City, New York', 'Oregon']                                  1
Length: 706, dtype: int64

# sort by genre 

In [ ]:
df = df.replace({'genres': {np.nan: ['Ignore this']}})

# df2['genres'] = df2['genres'].apply(eval)
def to_1D(series):
    return pd.Series([x for x in series])

to_1D(df["genres"]).value_counts()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

plt.title('', fontsize=16)
plt.xlim([-40,40])
plt.xlabel('')
plt.ylabel('')
plt.legend()
plt.savefig('.png')